This notebook is to demonstrate cleaning
 of big data with pyspark


In [2]:
!pip install pyspark

In [5]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np




Data cleaning with spark

Data cleaning is preparing data for use in processing pipeline

This can be reformatting, removing incomplete and performing calculations

Performance and organising


Spark is scalable and powerful for large data

Spark schemas help validate the format of the data frame

Can contain many types of data types

Can filter out data

In [6]:
# Schemas improve read performance
# csv don't have a predefined schema but usiing a predefined schema reduces the need to infer and improves peformance

#StructField('field_name', DataType(), nullable)
from pyspark.sql.types import *
# schema example
peopleSchema = StructType([
  #define each field in the schema
  StructField('name', StringType(), True),
  StructField('age', IntegerType(), True),
  StructField('city', StringType(), True)
])


Spark data frames are immutable, this means they can’t be changed after initialisation

Recreated if resigned
This helps share across partitions


Every time a variable is overwritten it destroys the variable and recreates it with the transformation. It does this so well with lazy processing it won’t do anything until it needs to for a process. This is called Lazy processing

This basically updates the instructions before running

In [7]:
spark = SparkSession.builder \
    .appName("MySparkSession") \
    .getOrCreate()

In [8]:
os.listdir()

['DallasCouncilVotes.csv.gz',
 'DallasCouncilVoters.csv.gz',
 'AA_DFW_2014_Departures_Short.csv.gz',
 'AA_DFW_2015_Departures_Short.csv.gz',
 'AA_DFW_2016_Departures_Short.csv.gz',
 'AA_DFW_2017_Departures_Short.csv.gz',
 'annotation.tar',
 'filename.parquet',
 'filename2.parquet',
 'AA_DFW_ALL.parquet',
 'output.json',
 'annotations.csv.gz',
 'dogs.csv.gz',
 'Annotation']

In PySpark, pyspark.sql.functions (often imported as F) is a module that provides a wide variety of functions specifically for DataFrame operations. These functions are optimized to perform data transformations, aggregations, and computations directly on columns within a Spark DataFrame, which helps improve performance by minimizing Python code execution in favor of Spark's native processing.


In [9]:
from pyspark.sql import functions as F


In [10]:
# Load the CSV file
aa_dfw_df = spark.read.format('csv').options(Header=True).load('AA_DFW_2017_Departures_Short.csv.gz')

# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))

# Show the DataFrame
aa_dfw_df.show()

+-----------------+-------------+-------------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-------------------+-----------------------------+-------+
|       01/01/2017|         0005|                HNL|                          537|    hnl|
|       01/01/2017|         0007|                OGG|                          498|    ogg|
|       01/01/2017|         0037|                SFO|                          241|    sfo|
|       01/01/2017|         0043|                DTW|                          134|    dtw|
|       01/01/2017|         0051|                STL|                           88|    stl|
|       01/01/2017|         0060|                MIA|                          149|    mia|
|       01/01/2017|         0071|                LAX|                          203|    lax|
|       01/01/2017|         0074|                MEM|                           

Parquet is a compressed columnar data format used by any Hadoop based system

Supports predicate pushdown and automatically stores schema info

Predicate push down in essence is ordering tasks to do the least amount of work

Parquet are a binary file format and can only be used by specific tools

Example read and write to parquet files:

In [11]:
import os
import shutil

file_path = 'filename.parquet'

# Check if path exists
if os.path.exists(file_path):
    # If it's a directory (e.g., parquet format often outputs a folder), use shutil.rmtree
    if os.path.isdir(file_path):
        shutil.rmtree(file_path)
    else:  # If it's a file, use os.remove
        os.remove(file_path)



In [12]:
aa_dfw_df.write.format('parquet').save('filename.parquet')

In [13]:
df = spark.read.parquet('filename.parquet')

In [14]:
file_path = 'filename2.parquet'

# Check if path exists
if os.path.exists(file_path):
    # If it's a directory (e.g., parquet format often outputs a folder), use shutil.rmtree
    if os.path.isdir(file_path):
        shutil.rmtree(file_path)
    else:  # If it's a file, use os.remove
        os.remove(file_path)

In [15]:
df.write.format('parquet').save('filename2.parquet')

In [16]:
df.columns

['Date (MM/DD/YYYY)',
 'Flight Number',
 'Destination Airport',
 'Actual elapsed time (Minutes)',
 'airport']

In [17]:


df = df.withColumnRenamed('Actual elapsed time (Minutes)','duration')
df.createOrReplaceTempView('flights')
df.show()

+-----------------+-------------+-------------------+--------+-------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|duration|airport|
+-----------------+-------------+-------------------+--------+-------+
|       01/01/2017|         0005|                HNL|     537|    hnl|
|       01/01/2017|         0007|                OGG|     498|    ogg|
|       01/01/2017|         0037|                SFO|     241|    sfo|
|       01/01/2017|         0043|                DTW|     134|    dtw|
|       01/01/2017|         0051|                STL|      88|    stl|
|       01/01/2017|         0060|                MIA|     149|    mia|
|       01/01/2017|         0071|                LAX|     203|    lax|
|       01/01/2017|         0074|                MEM|      76|    mem|
|       01/01/2017|         0081|                DEN|     123|    den|
|       01/01/2017|         0089|                SLC|     161|    slc|
|       01/01/2017|         0096|                STL|      84|    stl|
|     

In [18]:
df_query = spark.sql('SELECT * FROM flights WHERE duration < 120')
df_query.show()

+-----------------+-------------+-------------------+--------+-------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|duration|airport|
+-----------------+-------------+-------------------+--------+-------+
|       01/01/2017|         0051|                STL|      88|    stl|
|       01/01/2017|         0074|                MEM|      76|    mem|
|       01/01/2017|         0096|                STL|      84|    stl|
|       01/01/2017|         0190|                SAT|      76|    sat|
|       01/01/2017|         0229|                KOA|       0|    koa|
|       01/01/2017|         0232|                ATL|     117|    atl|
|       01/01/2017|         0299|                TUL|      59|    tul|
|       01/01/2017|         0315|                SAT|      53|    sat|
|       01/01/2017|         0320|                SAT|      56|    sat|
|       01/01/2017|         0326|                STL|      91|    stl|
|       01/01/2017|         0357|                SAT|      57|    sat|
|     

In [19]:
os.listdir()

['DallasCouncilVotes.csv.gz',
 'DallasCouncilVoters.csv.gz',
 'AA_DFW_2014_Departures_Short.csv.gz',
 'AA_DFW_2015_Departures_Short.csv.gz',
 'AA_DFW_2016_Departures_Short.csv.gz',
 'AA_DFW_2017_Departures_Short.csv.gz',
 'annotation.tar',
 'AA_DFW_ALL.parquet',
 'output.json',
 'annotations.csv.gz',
 'dogs.csv.gz',
 'Annotation',
 'filename.parquet',
 'filename2.parquet']

In [20]:
df1 = spark.read.format('csv').options(Header=True).load('AA_DFW_2017_Departures_Short.csv.gz')
df2 = spark.read.format('csv').options(Header=True).load('AA_DFW_2016_Departures_Short.csv.gz')

In [21]:
# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())


df1 Count: 139358
df2 Count: 140604


In [22]:
# Combine the DataFrames into one
df3 = df1.union(df2)


In [23]:
# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

In [24]:
# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())


279962


In [25]:

# Read the Parquet file into flights_df
flights_df = spark.read.parquet('AA_DFW_ALL.parquet')

flights_df = flights_df.withColumnRenamed('Actual elapsed time (Minutes)','flight_duration')

flights_df.show()

+-----------------+-------------+-------------------+---------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|flight_duration|
+-----------------+-------------+-------------------+---------------+
|       01/01/2016|         0005|                HNL|            529|
|       01/01/2016|         0007|                OGG|            512|
|       01/01/2016|         0025|                PHL|            161|
|       01/01/2016|         0037|                SFO|            259|
|       01/01/2016|         0043|                DTW|            157|
|       01/01/2016|         0060|                MIA|            144|
|       01/01/2016|         0071|                LAS|            165|
|       01/01/2016|         0079|                SLC|            153|
|       01/01/2016|         0081|                TUS|            133|
|       01/01/2016|         0098|                SMF|            207|
|       01/01/2016|         0103|                SJC|            231|
|       01/01/2016| 

In [26]:
# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)



The average flight time is: 151


Common spark data frame transformations

.filer()
.where
.withColumn
.drop

You can use .filter / .where to clean data:

Df.filter(df[‘column’].isNotNull())
df.filter(df.column > n)
df.where(df[‘col’].contins(‘string’))
df.where(~ df.column.isNull())
The ~ means the negation so opposite


Most cleaning functions are in pyspark.sql.functions which is typically aliased as F


In [27]:
df.show()

+-----------------+-------------+-------------------+--------+-------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|duration|airport|
+-----------------+-------------+-------------------+--------+-------+
|       01/01/2017|         0005|                HNL|     537|    hnl|
|       01/01/2017|         0007|                OGG|     498|    ogg|
|       01/01/2017|         0037|                SFO|     241|    sfo|
|       01/01/2017|         0043|                DTW|     134|    dtw|
|       01/01/2017|         0051|                STL|      88|    stl|
|       01/01/2017|         0060|                MIA|     149|    mia|
|       01/01/2017|         0071|                LAX|     203|    lax|
|       01/01/2017|         0074|                MEM|      76|    mem|
|       01/01/2017|         0081|                DEN|     123|    den|
|       01/01/2017|         0089|                SLC|     161|    slc|
|       01/01/2017|         0096|                STL|      84|    stl|
|     

In [28]:
import pyspark.sql.functions as F
df=df.withColumn('newCol',F.upper('airport'))

df=df.withColumn('date', F.split('Date (MM/DD/YYYY)','/')[2])

#cast is a function that changes data types
df = df.withColumn('year', df['date'].cast(IntegerType()))


In [29]:
df.show()

+-----------------+-------------+-------------------+--------+-------+------+----+----+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|duration|airport|newCol|date|year|
+-----------------+-------------+-------------------+--------+-------+------+----+----+
|       01/01/2017|         0005|                HNL|     537|    hnl|   HNL|2017|2017|
|       01/01/2017|         0007|                OGG|     498|    ogg|   OGG|2017|2017|
|       01/01/2017|         0037|                SFO|     241|    sfo|   SFO|2017|2017|
|       01/01/2017|         0043|                DTW|     134|    dtw|   DTW|2017|2017|
|       01/01/2017|         0051|                STL|      88|    stl|   STL|2017|2017|
|       01/01/2017|         0060|                MIA|     149|    mia|   MIA|2017|2017|
|       01/01/2017|         0071|                LAX|     203|    lax|   LAX|2017|2017|
|       01/01/2017|         0074|                MEM|      76|    mem|   MEM|2017|2017|
|       01/01/2017|         0081

With ArrayType() you can use .size() to get length of array

.getItem(index) to get the item at the given index

In [30]:
os.listdir()

['DallasCouncilVotes.csv.gz',
 'DallasCouncilVoters.csv.gz',
 'AA_DFW_2014_Departures_Short.csv.gz',
 'AA_DFW_2015_Departures_Short.csv.gz',
 'AA_DFW_2016_Departures_Short.csv.gz',
 'AA_DFW_2017_Departures_Short.csv.gz',
 'annotation.tar',
 'output.json',
 'annotations.csv.gz',
 'dogs.csv.gz',
 'Annotation',
 'filename.parquet',
 'filename2.parquet',
 'AA_DFW_ALL.parquet']

In [31]:
voter_df = spark.read.csv('DallasCouncilVoters.csv.gz',header=True)
voter_df.show()

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|
|02/08/2017|Councilmember|       Scott Griggs|
|02/08/2017|Councilmember|   B. Adam  McGough|
|02/08/2017|Councilmember|       Lee Kleinman|
|02/08/2017|Councilmember|      Sandy Greyson|
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|
|02/08/2017|Councilmember| Rickey D. Callahan|
|01/11/2017|Councilmember|  Jennifer S. Gates|
|04/25/2018|C

In [32]:
# Show the distinct VOTER_NAME entries
voter_df.select(voter_df['VOTER_NAME']).distinct().show(40, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|VOTER_NAME                                                                                                                                                                                                                                                                                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [33]:
# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

In [34]:
# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains('_'))

In [35]:
# Show the distinct VOTER_NAME entries again
voter_df.select('VOTER_NAME').distinct().show(40, truncate=False)

+-------------------+
|VOTER_NAME         |
+-------------------+
|Tennell Atkins     |
|Scott Griggs       |
|Scott  Griggs      |
|Sandy Greyson      |
|Michael S. Rawlings|
|Kevin Felder       |
|Adam Medrano       |
|Casey  Thomas      |
|Mark  Clayton      |
|Casey Thomas       |
|Sandy  Greyson     |
|Mark Clayton       |
|Jennifer S.  Gates |
|Tiffinni A. Young  |
|B. Adam  McGough   |
|Omar Narvaez       |
|Philip T. Kingston |
|Rickey D. Callahan |
|Dwaine R. Caraway  |
|Philip T.  Kingston|
|Jennifer S. Gates  |
|Lee M. Kleinman    |
|Monica R. Alonzo   |
|Rickey D.  Callahan|
|Carolyn King Arnold|
|Erik Wilson        |
|Lee Kleinman       |
+-------------------+



In [36]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

In [37]:
# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

In [38]:
# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))


/usr/local/lib/python3.10/dist-packages/pyspark/sql/column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(



Using if then else will be computationally heavy

Pyspark uses .when allows for if then

df.select(df.column, F.when(df.column == condition, action)

df.select(df.column, .when(df.column == condition, action).otherwise(action2’)

In [39]:
# Add a column to voter_df for any voter with the title **Councilmember**
voter_df = voter_df.withColumn('random_val', F.when(voter_df.TITLE == 'Councilmember', F.rand()))
# Show some of the DataFrame rows, noting whether the when clause worked
voter_df.show()


+----------+-------------+-------------------+--------------------+----------+---------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|              splits|first_name|last_name|         random_val|
+----------+-------------+-------------------+--------------------+----------+---------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|    Gates| 0.9304268866383663|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip| Kingston| 0.7679052085644473|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael| Rawlings|               NULL|
|02/08/2017|Councilmember|       Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|0.38807990802816905|
|02/08/2017|Councilmember|       Casey Thomas|     [Casey, Thomas]|     Casey|   Thomas| 0.8449889659586487|
|02/08/2017|Councilmember|Carolyn King Arnold|[Carolyn, King, A...|   Carolyn|   Arnold|0.08781713706161498|
|02/08/2017|Council

In [40]:
# we can update thje earlier code with more conditions:

voter_df = voter_df.withColumn(
    'random_val',
    F.when(voter_df.TITLE == 'Councilmember', F.rand())
     .when(voter_df.TITLE == 'Mayor', F.lit(2))  # Use F.lit(2) to specify a literal value
     .otherwise(F.lit(0))  # F.lit(0) as a default for other titles
)

# Show some of the DataFrame rows
voter_df.show()


+----------+-------------+-------------------+--------------------+----------+---------+--------------------+
|      DATE|        TITLE|         VOTER_NAME|              splits|first_name|last_name|          random_val|
+----------+-------------+-------------------+--------------------+----------+---------+--------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|    Gates| 0.06338505385631621|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip| Kingston|  0.5036546127818926|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael| Rawlings|                 2.0|
|02/08/2017|Councilmember|       Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|  0.9760448844315016|
|02/08/2017|Councilmember|       Casey Thomas|     [Casey, Thomas]|     Casey|   Thomas| 0.32499250953046455|
|02/08/2017|Councilmember|Carolyn King Arnold|[Carolyn, King, A...|   Carolyn|   Arnold|  0.9764441790164471|
|02/08/201

In [41]:
# Use the .filter() clause with random_val
voter_df.filter(voter_df.random_val==0).show()

+----------+--------------------+-----------------+--------------------+----------+---------+----------+
|      DATE|               TITLE|       VOTER_NAME|              splits|first_name|last_name|random_val|
+----------+--------------------+-----------------+--------------------+----------+---------+----------+
|04/25/2018|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|
|04/25/2018|       Mayor Pro Tem|Dwaine R. Caraway|[Dwaine, R., Cara...|    Dwaine|  Caraway|       0.0|
|06/20/2018|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|
|06/20/2018|       Mayor Pro Tem|Dwaine R. Caraway|[Dwaine, R., Cara...|    Dwaine|  Caraway|       0.0|
|06/20/2018|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|
|06/20/2018|       Mayor Pro Tem|Dwaine R. Caraway|[Dwaine, R., Cara...|    Dwaine|  Caraway|       0.0|
|08/15/2018|Deputy Mayor Pro Tem|     Adam Medrano|    

Udf's are user defined functions logic, once written a method is called via functions.udf and can be called as a normal function


You will need to define a function EG

Def reverseString(myStr)
	return myStr[::-1]


udfreverseString = udf(reverseString, StringType())

df = df.withColumn(‘newColumn’,udfreverseString(df.column))


In [42]:
def getFirstAndMiddle(names):
  # Return a space separated string of names
  return ' '.join(names[:-1])

In [43]:
# Define the method as a UDF
udfFirstAndMiddle = F.udf(getFirstAndMiddle, StringType())


In [44]:
# Create a new column using your UDF
voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df.splits))

# Show the DataFrame
voter_df.show()

+----------+-------------+-------------------+--------------------+----------+---------+--------------------+---------------------+
|      DATE|        TITLE|         VOTER_NAME|              splits|first_name|last_name|          random_val|first_and_middle_name|
+----------+-------------+-------------------+--------------------+----------+---------+--------------------+---------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|    Gates| 0.06338505385631621|          Jennifer S.|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip| Kingston|  0.5036546127818926|            Philip T.|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael| Rawlings|                 2.0|           Michael S.|
|02/08/2017|Councilmember|       Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|  0.9760448844315016|                 Adam|
|02/08/2017|Councilmember|       Casey Thomas|     [Casey, Thomas]|     Case

In [45]:
# Select all the unique council voters
voter_df = voter_df.select(voter_df["VOTER_NAME"]).distinct()

# Count the rows in voter_df
print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())



There are 27 rows in the voter_df DataFrame.



monotonically_increasing_id() in

Ensures Unique IDs Across Partitions: When working in a distributed environment, each partition in Spark may process its portion of the data independently. monotonically_increasing_id() assigns unique IDs within and across partitions, ensuring no two rows have the same ID even if they’re processed separately.

Monotonic but Not Consecutive: The function generates IDs that are unique and increasing but not necessarily consecutive across partitions. This is because each partition receives a distinct range of values. This behavior protects against conflicts and ensures IDs are unique without strict sequential ordering.

Useful for Row Numbering: monotonically_increasing_id() is often used when you need a unique identifier or row number but don’t require strict sequential ordering (such as for deduplication, merging, or creating unique keys).

Important Characteristics
Distributed Safe: The IDs generated are unique across the distributed partitions, which is critical for large-scale datasets.
Non-consecutive IDs: Because it’s partition-safe, the IDs will not be strictly sequential. Instead, each partition starts its numbering from a large offset.


In [46]:

# Add a ROW_ID
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the rows with 10 highest IDs in the set
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)

+-------------------+------+
|         VOTER_NAME|ROW_ID|
+-------------------+------+
|       Lee Kleinman|    26|
|        Erik Wilson|    25|
|Carolyn King Arnold|    24|
|Rickey D.  Callahan|    23|
|   Monica R. Alonzo|    22|
|    Lee M. Kleinman|    21|
|  Jennifer S. Gates|    20|
|Philip T.  Kingston|    19|
|  Dwaine R. Caraway|    18|
| Rickey D. Callahan|    17|
+-------------------+------+
only showing top 10 rows



In [47]:
# Print the number of partitions in each DataFrame
print("\nThere are %d partitions in the voter_df DataFrame.\n" % voter_df.rdd.getNumPartitions())


# Add a ROW_ID field to each DataFrame
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())


There are 1 partitions in the voter_df DataFrame.



In [48]:
# Determine the highest ROW_ID and save it in previous_max_ID
previous_max_ID = voter_df.select('ROW_ID').rdd.max()[0] + 1

In [49]:

# Add a ROW_ID column to voter_df_april starting at the desired value
voter_df= voter_df.withColumn('ROW_ID2',  F.monotonically_increasing_id()+ previous_max_ID)

Caching:
Caching stores DataFrames in memory or on disk

This speeds up later actions / transformations
This reduces resource usage

Data may not fit in memory
Local disk based caching may not improveme performance
Cached objects may not be available

Cache in fast ssd or nvme

Intermediate files lie parquet

Stop caching when done

In pyspark .cache() is used to cache

df = df.cache() - caches
df.is_cached - checks if cached
df.unpersist() - removes cache

In [50]:
departures_df = spark.read.csv('AA_DFW_*', header = True)

In [51]:
import time
start_time = time.time()

# Add caching to the unique rows in departures_df
departures_df = departures_df.distinct().cache()

# Count the unique rows in departures_df, noting how long the operation takes
print("Counting %d rows took %f seconds" % (departures_df.count(), time.time() - start_time))

Counting 590726 rows took 21.166335 seconds


In [52]:
# Count the rows again, noting the variance in time of a cached DataFrame
start_time = time.time()
print("Counting %d rows again took %f seconds" % (departures_df.count(), time.time() - start_time))

Counting 590726 rows again took 3.315447 seconds


In [53]:
# Determine if departures_df is in the cache
print("Is departures_df cached?: %s" % departures_df.is_cached)

Is departures_df cached?: True


In [54]:
print("Removing departures_df from cache")

# Remove departures_df from the cache
departures_df.unpersist()
# Check the cache status again
print("Is departures_df cached?: %s" % departures_df.is_cached)

Removing departures_df from cache
Is departures_df cached?: False


Spark consists of a driver process and multiple worker processes.

The driver orchestrates the application, assigning tasks to workers.
Workers execute these tasks independently on data partitions and return results to the driver.
This setup enables distributed processing across clusters, making Spark highly efficient for big data.

split -l rows -d filename prefix-

You can read in and write to parquet to split things up

In [55]:
os.listdir()

['DallasCouncilVotes.csv.gz',
 'DallasCouncilVoters.csv.gz',
 'AA_DFW_2014_Departures_Short.csv.gz',
 'AA_DFW_2015_Departures_Short.csv.gz',
 'AA_DFW_2016_Departures_Short.csv.gz',
 'AA_DFW_2017_Departures_Short.csv.gz',
 'annotation.tar',
 'output.json',
 'annotations.csv.gz',
 'dogs.csv.gz',
 'Annotation',
 'filename.parquet',
 'filename2.parquet',
 'AA_DFW_ALL.parquet']

In [56]:
# # Import the full and split files into DataFrames
# full_df = spark.read.csv('departures_full.txt.gz')
# split_df = spark.read.csv('departures_*.txt.gz')


When configuring spark you can use commands

Spark.conf.get(configuration_name)

spark.cong.set(configuration_name)


The cluster types are: single node, standalone and managed

Due to the responsibility of the driver it is recommended to double he driver memory in comparison to worker

More workers over larger workers

Fast local storage is very useful with caching



In [57]:
# Name of the Spark application instance
app_name = spark.conf.get('spark.app.name')

# Driver TCP port
driver_tcp_port = spark.conf.get('spark.driver.port')

# Number of join partitions
num_partitions = spark.conf.get('spark.sql.shuffle.partitions')

# Show the results
print("Name: %s" % app_name)
print("Driver TCP port: %s" % driver_tcp_port)
print("Number of partitions: %s" % num_partitions)

Name: MySparkSession
Driver TCP port: 42247
Number of partitions: 200


In [58]:
# Store the number of partitions in variable
before = departures_df.rdd.getNumPartitions()

# Configure Spark to use 500 partitions
spark.conf.set('spark.sql.shuffle.partitions', 500)

In [59]:
# Load all files matching the wildcard pattern and remove duplicates
departures_df = spark.read.csv('AA_DFW_*', header = True).distinct()

# Check the partition count after repartitioning
after = departures_df.rdd.getNumPartitions()
print("Partition count after change: %d" % after)


Partition count after change: 2


In [60]:
# .explain() lets you know what is going on to show what is ran
departures_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(2) HashAggregate(keys=[Date (MM/DD/YYYY)#907, Flight Number#908, Destination Airport#909, Actual elapsed time (Minutes)#910], functions=[])
   +- AQEShuffleRead coalesced
      +- ShuffleQueryStage 0
         +- Exchange hashpartitioning(Date (MM/DD/YYYY)#907, Flight Number#908, Destination Airport#909, Actual elapsed time (Minutes)#910, 500), ENSURE_REQUIREMENTS, [plan_id=1076]
            +- *(1) HashAggregate(keys=[Date (MM/DD/YYYY)#907, Flight Number#908, Destination Airport#909, Actual elapsed time (Minutes)#910], functions=[])
               +- FileScan csv [Date (MM/DD/YYYY)#907,Flight Number#908,Destination Airport#909,Actual elapsed time (Minutes)#910] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(5 paths)[file:/content/drive/My Drive/cleaning_data_with_pyspark/AA_DFW_2014_De..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Date (MM/DD/YYYY):string,Flight 


Shuffling is the moving of fragments to workers to complete tasks and hides complexity
Can be show to complete but lowers throughput
Necessary but requires minimising

repartition(num_partitions)
Does a full shuffle and requires a full shuffle

.coalesce(num_partitions)
This consolidates without a full reshuffle

.join() causes shuffle operations
.broadcast() works as an alternative

Provides a copy of an object to each worker

Prevents undue shuffles and speeds up broadcast operations


In [61]:
votes_df = spark.read.csv('DallasCouncilVotes.csv.gz',header=True)
votes_df.show()

+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+
|      DATE|AGENDA_ITEM_NUMBER|  ITEM_TYPE|DISTRICT|        TITLE|         VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCRIPTION|         AGENDA_ID|             VOTE_ID|
+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+
|02/08/2017|                 1|     AGENDA|      13|Councilmember|  Jennifer S. Gates|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|
|02/08/2017|                 1|     AGENDA|      14|Councilmember| Philip T. Kingston|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|
|02/08/2017|                 1|     AGENDA|      15|        Mayor|Michael S. Rawlings|      N/A|  NO

In [62]:
voter_df.show()

+-------------------+------+-------+
|         VOTER_NAME|ROW_ID|ROW_ID2|
+-------------------+------+-------+
|   B. Adam  McGough|     0|     27|
| Philip T. Kingston|     1|     28|
|   Monica R. Alonzo|     2|     29|
|      Mark  Clayton|     3|     30|
|Michael S. Rawlings|     4|     31|
|       Kevin Felder|     5|     32|
|       Adam Medrano|     6|     33|
|       Omar Narvaez|     7|     34|
|  Dwaine R. Caraway|     8|     35|
|       Mark Clayton|     9|     36|
| Rickey D. Callahan|    10|     37|
|    Lee M. Kleinman|    11|     38|
|       Casey Thomas|    12|     39|
|        Erik Wilson|    13|     40|
|     Sandy  Greyson|    14|     41|
|     Tennell Atkins|    15|     42|
|      Sandy Greyson|    16|     43|
|  Tiffinni A. Young|    17|     44|
|Carolyn King Arnold|    18|     45|
|       Scott Griggs|    19|     46|
+-------------------+------+-------+
only showing top 20 rows



In [63]:
from pyspark.sql.functions import broadcast

combined_df = votes_df.join(broadcast(voter_df))


In [64]:
os.listdir()

['DallasCouncilVotes.csv.gz',
 'DallasCouncilVoters.csv.gz',
 'AA_DFW_2014_Departures_Short.csv.gz',
 'AA_DFW_2015_Departures_Short.csv.gz',
 'AA_DFW_2016_Departures_Short.csv.gz',
 'AA_DFW_2017_Departures_Short.csv.gz',
 'annotation.tar',
 'output.json',
 'annotations.csv.gz',
 'dogs.csv.gz',
 'Annotation',
 'filename.parquet',
 'filename2.parquet',
 'AA_DFW_ALL.parquet']

In [65]:
votes_df.show()

+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+
|      DATE|AGENDA_ITEM_NUMBER|  ITEM_TYPE|DISTRICT|        TITLE|         VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCRIPTION|         AGENDA_ID|             VOTE_ID|
+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+-----------------------+------------------+--------------------+
|02/08/2017|                 1|     AGENDA|      13|Councilmember|  Jennifer S. Gates|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|
|02/08/2017|                 1|     AGENDA|      14|Councilmember| Philip T. Kingston|      N/A|  NO ACTION NEEDED|          Call to Order|020817__Special__1|020817__Special__...|
|02/08/2017|                 1|     AGENDA|      15|        Mayor|Michael S. Rawlings|      N/A|  NO

In [66]:
# Join the flights_df and airports_df DataFrames using broadcasting
broadcast_df = combined_df = votes_df.join(broadcast(voter_df), votes_df["ITEM_TYPE"] >1 )
broadcast_df.show()

+----+------------------+---------+--------+-----+----------+---------+------------------+-----------------------+---------+-------+----------+------+-------+
|DATE|AGENDA_ITEM_NUMBER|ITEM_TYPE|DISTRICT|TITLE|VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCRIPTION|AGENDA_ID|VOTE_ID|VOTER_NAME|ROW_ID|ROW_ID2|
+----+------------------+---------+--------+-----+----------+---------+------------------+-----------------------+---------+-------+----------+------+-------+
+----+------------------+---------+--------+-----+----------+---------+------------------+-----------------------+---------+-------+----------+------+-------+



In [67]:
# Show the query plan and compare against the original
broadcast_df.explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastNestedLoopJoin BuildRight, Inner
   :- Filter (isnotnull(ITEM_TYPE#939) AND (cast(ITEM_TYPE#939 as int) > 1))
   :  +- FileScan csv [DATE#937,AGENDA_ITEM_NUMBER#938,ITEM_TYPE#939,DISTRICT#940,TITLE#941,VOTER NAME#942,VOTE CAST#943,FINAL ACTION TAKEN#944,AGENDA ITEM DESCRIPTION#945,AGENDA_ID#946,VOTE_ID#947] Batched: false, DataFilters: [isnotnull(ITEM_TYPE#939), (cast(ITEM_TYPE#939 as int) > 1)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/content/drive/My Drive/cleaning_data_with_pyspark/DallasCouncilV..., PartitionFilters: [], PushedFilters: [IsNotNull(ITEM_TYPE)], ReadSchema: struct<DATE:string,AGENDA_ITEM_NUMBER:string,ITEM_TYPE:string,DISTRICT:string,TITLE:string,VOTER ...
   +- BroadcastExchange IdentityBroadcastMode, [plan_id=1441]
      +- Project [VOTER_NAME#375, ROW_ID#609L, (monotonically_increasing_id() + 27) AS ROW_ID2#614L]
         +- HashAggregate(keys=[VOTER_NAME#375], functions=[])
       

In [68]:
start_time = time.time()
# Count the number of rows in the normal DataFrame
normal_count = votes_df.count()
normal_duration = time.time() - start_time

In [69]:
start_time = time.time()
# Count the number of rows in the broadcast DataFrame
broadcast_count = broadcast_df.count()
broadcast_duration = time.time() - start_time

In [70]:
# Print the counts and the duration of the tests
print("Normal count:\t\t%d\tduration: %f" % (normal_count, normal_duration))
print("Broadcast count:\t%d\tduration: %f" % (broadcast_count, broadcast_duration))


Normal count:		44625	duration: 0.437175
Broadcast count:	0	duration: 2.636467


In [71]:
os.listdir()

['DallasCouncilVotes.csv.gz',
 'DallasCouncilVoters.csv.gz',
 'AA_DFW_2014_Departures_Short.csv.gz',
 'AA_DFW_2015_Departures_Short.csv.gz',
 'AA_DFW_2016_Departures_Short.csv.gz',
 'AA_DFW_2017_Departures_Short.csv.gz',
 'annotation.tar',
 'output.json',
 'annotations.csv.gz',
 'dogs.csv.gz',
 'Annotation',
 'filename.parquet',
 'filename2.parquet',
 'AA_DFW_ALL.parquet']

In [72]:
# paImport the data to a DataFrame
departures_df = spark.read.csv('AA_DFW_2015_Departures_Short.csv.gz', header=True)

In [73]:
departures_df = departures_df.withColumnRenamed('Actual elapsed time (Minutes)','duration')
# Remove any duration of 0
departures_df = departures_df.filter(departures_df[3] > 0)

In [74]:
departures_df.show()

+-----------------+-------------+-------------------+--------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|duration|
+-----------------+-------------+-------------------+--------+
|       01/01/2015|         0005|                HNL|     526|
|       01/01/2015|         0007|                OGG|     517|
|       01/01/2015|         0023|                SFO|     233|
|       01/01/2015|         0027|                LAS|     165|
|       01/01/2015|         0035|                HDN|     178|
|       01/01/2015|         0037|                SAN|     187|
|       01/01/2015|         0049|                SAN|     178|
|       01/01/2015|         0051|                SLC|     161|
|       01/01/2015|         0060|                MIA|     151|
|       01/01/2015|         0064|                JFK|     187|
|       01/01/2015|         0071|                SAN|     176|
|       01/01/2015|         0072|                MCO|     142|
|       01/01/2015|         0079|                SMF|  

In [75]:
# Add an ID column
departures_df = departures_df.withColumn('id', F.monotonically_increasing_id())


In [76]:
# Write the file out to JSON format
departures_df.write.json('output.json', mode='overwrite')

Reading in data parses the file,
When reading in with spark you can specify comment and it’s delimiter to remove them, and Header specifying whether a predefined header is true or false

You can use sep to specify the delimiter if not defined will assume ,

If the Datta can’t be separated it will be stored under column _c0 as default allowing you to properly handle nested separators

In [77]:
import pandas as pd

# Create a sample DataFrame
data = {
    "_c0": ["# This is a comment line", "# Another comment line", "ID|Annotation|Value", "1|Sample1|42", "2|Sample2|56", "# Comment line in the middle", "3|Sample3|78", "4|Sample4|65", "5|Sample5|89"]
}
df = pd.DataFrame(data)

# Save the DataFrame to a gzipped CSV
df.to_csv("annotations.csv.gz", index=False, header=False, sep='|', compression="gzip")


In [78]:
# when importing you can define annotations so they're ignored

# Load the gzipped file
annotations_df = spark.read.csv('annotations.csv.gz', sep='|')
full_count = annotations_df.count()



# Load again without comments
no_comments_df = spark.read.csv('annotations.csv.gz', sep='|', comment='#')
no_comments_count = no_comments_df.count()

print("Full count:  %d\nRemaining count: %d" % (full_count, no_comments_count))


Full count:  9
Remaining count: 6


In [79]:
import pandas as pd
import random

# Generate sample data with 500 rows and varying numbers of tab-separated fields
data = []
for i in range(1, 501):  # Create 500 rows
    # Randomly choose the number of fields (between 3 and 6)
    fields_count = random.choice([3, 4, 5, 6])

    # Define values for the fields
    folder = f"Folder_{random.randint(1, 10)}" if fields_count >= 1 else ""
    filename = f"file_{i}.jpg" if fields_count >= 2 else ""
    width = str(random.choice([640, 800, 1024, 1280, 1920])) if fields_count >= 3 else ""
    height = str(random.choice([480, 600, 768, 720, 1080])) if fields_count >= 4 else ""
    extra_info = f"ExtraInfo_{random.randint(1, 100)}" if fields_count >= 5 else ""

    # Combine fields into a tab-separated string
    row = '\t'.join([folder, filename, width, height, extra_info]).strip()
    data.append([row])

# Convert to DataFrame
df = pd.DataFrame(data, columns=["_c0"])

# Save to a gzipped CSV file
df.to_csv("annotations.csv.gz", index=False, header=False, compression="gzip", sep='\t')


In [80]:
annotations_df = spark.read.csv('annotations.csv.gz', sep='\t')
initial_count = annotations_df.count()
# Split the first column on tab and store in tmp_fields
tmp_fields = F.split(annotations_df['_c0'], '\t')

# Create the colcount column
annotations_df = annotations_df.withColumn('colcount', F.size(tmp_fields))

# Filter out rows with fewer than 5 fields
annotations_df_filtered = annotations_df.filter(~ (annotations_df["colcount"] < 5))

# Show results
annotations_df_filtered.show()

+--------------------+--------+
|                 _c0|colcount|
+--------------------+--------+
|Folder_4\tfile_2....|       5|
|Folder_3\tfile_6....|       5|
|Folder_3\tfile_7....|       5|
|Folder_9\tfile_8....|       5|
|Folder_2\tfile_9....|       5|
|Folder_3\tfile_10...|       5|
|Folder_3\tfile_15...|       5|
|Folder_1\tfile_16...|       5|
|Folder_4\tfile_17...|       5|
|Folder_4\tfile_18...|       5|
|Folder_7\tfile_19...|       5|
|Folder_10\tfile_2...|       5|
|Folder_1\tfile_22...|       5|
|Folder_5\tfile_26...|       5|
|Folder_9\tfile_27...|       5|
|Folder_8\tfile_28...|       5|
|Folder_1\tfile_29...|       5|
|Folder_5\tfile_30...|       5|
|Folder_10\tfile_3...|       5|
|Folder_6\tfile_32...|       5|
+--------------------+--------+
only showing top 20 rows



In [81]:
# Count the number of rows
final_count = annotations_df_filtered.count()
print("Initial count: %d\nFinal count: %d" % (initial_count, final_count))


Initial count: 500
Final count: 246


In [82]:
# Split the content of _c0 on the tab character (aka, '\t')
split_cols = F.split(annotations_df['_c0'], '\t')

# Add the columns folder, filename, width, and height
split_df = annotations_df.withColumn('folder', split_cols.getItem(0))
split_df = split_df.withColumn('filename', split_cols.getItem(1))
split_df = split_df.withColumn('width', split_cols.getItem(2))
split_df = split_df.withColumn('height', split_cols.getItem(3))
# Add split_cols as a column
split_df = split_df.withColumn('split_cols',split_cols)

In [83]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType


# Step 1: Generate sample data
data = []
for i in range(1, 501):  # Create 500 rows
    folder = f"Folder_{random.randint(1, 10)}"
    filename = f"file_{i}.jpg"
    width = str(random.choice([640, 800, 1024, 1280, 1920]))
    height = str(random.choice([480, 600, 768, 720, 1080]))

    # Generate random "dog data" fields, between 1 and 3 extra fields
    dog_data_count = random.randint(1, 3)
    dog_data = [f"DogData_{j}" for j in range(1, dog_data_count + 1)]
    row = '\t'.join([folder, filename, width, height] + dog_data)
    data.append([row])

# Create DataFrame from sample data and save as a gzipped CSV
df = pd.DataFrame(data, columns=["_c0"])
df.to_csv("dogs.csv.gz", index=False, header=False, compression="gzip", sep='\t')

# Step 2: Load the DataFrame and apply transformations
split_df = spark.read.csv('dogs.csv.gz', sep='\t')
split_df = split_df.withColumnRenamed('_c0', 'content')


split_cols = F.split(split_df['content'], '\t')
split_df = split_df.withColumn('split_cols', split_cols)
split_df = split_df.withColumn('colcount', F.size(split_cols))
split_df_filtered = split_df.filter(F.size(split_cols) >= 5)

In [84]:
def retriever(cols, colcount):
  # Return a list of dog data
  return cols[4:colcount]

In [85]:
# Define the method as a UDF
udfRetriever = F.udf(retriever, ArrayType(StringType()))

In [86]:
# Create a new column using your UDF
split_df = split_df.withColumn('dog_list', udfRetriever(split_df.split_cols, split_df.colcount))

# Remove the original column, split_cols, and the colcount
split_df = split_df.drop('_c0').drop('split_cols').drop('colcount')

you can validate the output in various ways one of which is using joins other verify content

In [87]:
os.listdir()

['DallasCouncilVotes.csv.gz',
 'DallasCouncilVoters.csv.gz',
 'AA_DFW_2014_Departures_Short.csv.gz',
 'AA_DFW_2015_Departures_Short.csv.gz',
 'AA_DFW_2016_Departures_Short.csv.gz',
 'AA_DFW_2017_Departures_Short.csv.gz',
 'annotation.tar',
 'annotations.csv.gz',
 'dogs.csv.gz',
 'Annotation',
 'filename.parquet',
 'filename2.parquet',
 'AA_DFW_ALL.parquet',
 'output.json']

In [88]:
df1 = spark.read.csv('AA_DFW_2014_Departures_Short.csv.gz', header=True)
df2 = spark.read.csv('AA_DFW_2014_Departures_Short.csv.gz', header=True)

In [89]:
df1.show(5)

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2014|         0005|                HNL|                          519|
|       01/01/2014|         0007|                OGG|                          505|
|       01/01/2014|         0035|                SLC|                          174|
|       01/01/2014|         0043|                DTW|                          153|
|       01/01/2014|         0052|                PIT|                          137|
+-----------------+-------------+-------------------+-----------------------------+
only showing top 5 rows



In [90]:
# Reference columns with spaces using backticks
before = df1.count()
verify = df1.join(F.broadcast(df2), df1["`Destination Airport`"] == df2["`Destination Airport`"])


In [91]:
print(verify)

DataFrame[Date (MM/DD/YYYY): string, Flight Number: string, Destination Airport: string, Actual elapsed time (Minutes): string, Date (MM/DD/YYYY): string, Flight Number: string, Destination Airport: string, Actual elapsed time (Minutes): string]


In [92]:
print(df1.count() == before)
# validate rows stayed the same

True


In [93]:
os.listdir()

['DallasCouncilVotes.csv.gz',
 'DallasCouncilVoters.csv.gz',
 'AA_DFW_2014_Departures_Short.csv.gz',
 'AA_DFW_2015_Departures_Short.csv.gz',
 'AA_DFW_2016_Departures_Short.csv.gz',
 'AA_DFW_2017_Departures_Short.csv.gz',
 'annotation.tar',
 'annotations.csv.gz',
 'dogs.csv.gz',
 'Annotation',
 'filename.parquet',
 'filename2.parquet',
 'AA_DFW_ALL.parquet',
 'output.json']

In [94]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType
import random

# Define parameters
num_rows = 1000
valid_folders = [f"folder_{i}" for i in range(1, 21)]  # 20 unique folders
breeds = ['Beagle', 'Bulldog', 'Chihuahua', 'Dachshund', 'Poodle', 'Rottweiler', 'Schnauzer', 'Labrador']

# Generate valid_folders_df
valid_folders_df = spark.createDataFrame([(folder,) for folder in valid_folders], ["folder"])

# Generate split_df
split_data = []
for _ in range(num_rows):
    folder = random.choice(valid_folders)  # Randomly choose a folder
    breed = random.choice(breeds)  # Random breed
    start_x = random.randint(0, 100)  # Random start_x
    start_y = random.randint(0, 100)  # Random start_y
    end_x = start_x + random.randint(20, 80)  # Random end_x
    end_y = start_y + random.randint(20, 80)  # Random end_y
    width = random.randint(100, 800)  # Random image width
    height = random.randint(100, 800)  # Random image height

    dog_list = f"{breed},{start_x},{start_y},{end_x},{end_y}"
    split_data.append((folder, dog_list, width, height))

split_df = spark.createDataFrame(split_data, ["folder", "dog_list", "width", "height"])

# Show sample data
valid_folders_df.show(truncate=False)
split_df.show(truncate=False)

# Continue with the rest of your processing code...


+---------+
|folder   |
+---------+
|folder_1 |
|folder_2 |
|folder_3 |
|folder_4 |
|folder_5 |
|folder_6 |
|folder_7 |
|folder_8 |
|folder_9 |
|folder_10|
|folder_11|
|folder_12|
|folder_13|
|folder_14|
|folder_15|
|folder_16|
|folder_17|
|folder_18|
|folder_19|
|folder_20|
+---------+

+---------+-----------------------+-----+------+
|folder   |dog_list               |width|height|
+---------+-----------------------+-----+------+
|folder_13|Rottweiler,86,12,110,61|144  |330   |
|folder_16|Rottweiler,14,53,47,91 |735  |251   |
|folder_19|Poodle,28,79,83,154    |700  |275   |
|folder_11|Bulldog,46,0,107,67    |775  |466   |
|folder_7 |Schnauzer,17,96,75,132 |409  |458   |
|folder_14|Poodle,3,14,61,36      |754  |705   |
|folder_3 |Bulldog,45,66,111,89   |501  |789   |
|folder_9 |Beagle,8,58,35,89      |311  |284   |
|folder_2 |Dachshund,98,60,119,110|139  |159   |
|folder_17|Beagle,7,98,36,152     |457  |530   |
|folder_19|Chihuahua,73,95,140,117|528  |698   |
|folder_1 |Beagle,60,36,1

In [95]:
# Count the number of rows in split_df
split_count = split_df.count()


In [96]:
# Join the DataFrames
joined_df = split_df.join(F.broadcast(valid_folders_df), "folder")

In [97]:
# Compare the number of rows remaining
joined_count = joined_df.count()
print("Before: %d\nAfter: %d" % (split_count, joined_count))


Before: 1000
After: 1000


In [98]:
# Determine the row counts for each DataFrame
split_count = split_df.count()
joined_count = joined_df.count()

In [99]:
# Create a DataFrame containing the invalid rows
# left anti only keeps those in the left df with no match
invalid_df = split_df.join(F.broadcast(joined_df), 'folder', 'left_anti')

In [100]:
# Validate the count of the new DataFrame is as expected
invalid_count = invalid_df.count()
print(" split_df:\t%d\n joined_df:\t%d\n invalid_df: \t%d" % (split_count, joined_count, invalid_count))

 split_df:	1000
 joined_df:	1000
 invalid_df: 	0


In [101]:
# Determine the number of distinct folder rows removed
invalid_folder_count = invalid_df.select('folder').distinct().count()
print("%d distinct invalid folders found" % invalid_folder_count)

0 distinct invalid folders found


In [109]:
# Select the dog details and show 10 untruncated rows
print(joined_df.select('dog_list').show(10, truncate=False))

# Define a schema type for the details in the dog list
DogType = StructType([
    StructField("breed", StringType(), False),
    StructField("start_x", IntegerType(), False),
    StructField("start_y", IntegerType(), False),
    StructField("end_x", IntegerType(), False),
    StructField("end_y", IntegerType(), False)
])

+-----------------------+
|dog_list               |
+-----------------------+
|Rottweiler,86,12,110,61|
|Rottweiler,14,53,47,91 |
|Poodle,28,79,83,154    |
|Bulldog,46,0,107,67    |
|Schnauzer,17,96,75,132 |
|Poodle,3,14,61,36      |
|Bulldog,45,66,111,89   |
|Beagle,8,58,35,89      |
|Dachshund,98,60,119,110|
|Beagle,7,98,36,152     |
+-----------------------+
only showing top 10 rows

None
